# Process orders data
- 1.Ingest the data into the lakehouse -- bronze_orders
- 2.perform data quality checks and transform the data as required -- silver_orders_clean 
- 3.explode the items in the array orders object -- silver_orders

1.Ingest the data into the data lakehouse -- bronze_orders

In [0]:
create or refresh streaming table bronze_orders
comment "raw orders data ingested from the source system"
tblproperties("quality"="bronze")
as 

select *,
      _metadata.file_path as input_path,
      current_timestamp as ingested_timestamp
      from cloud_files(
        '/Volumes/circuitbox/landing/operational_data/orders/','json',
        map('cloudFiles.inferColumnTypes','true')
      );


In [0]:
create or refresh streaming table silver_orders_cleaned(
  constraint valid_customer_id expect(customer_id is not null) on violation fail update ,
  constraint valid_order_id expect(order_id is not null) on violation fail update,
  constraint valid_order_status expect(order_status in ('Pending','Shipped','Cancelled','Completed')),
  constraint valid_payment_method expect (payment_method in ('Credit Card','PayPal','Bank Transfer'))
)
comment "cleaned data for the silver_orders_cleaned table"
tblproperties("quality"="silver")
as
select order_id,
       customer_id,
       cast (order_timestamp  as timestamp) as order_timestamp,
       payment_method,
       items,
       order_status
       from stream(live.bronze_orders)

3.explode the items array from the order object -- silver_orders

In [0]:

create or refresh streaming table silver_orders
as 
select order_id,
      customer_id,
      order_timestamp,
      payment_method,
      order_status,
      item.item_id,
      item.name as item_name,
      item.price as item_price,
      item.quantity as item_quantity,
      item.category as item_category
      from
          (select order_id,
                customer_id,
                order_timestamp,
                payment_method,
                order_status,
                explode(items) as item
                from stream(live.silver_orders_cleaned))
